In [ ]:
import os
import requests
import json

In [ ]:
from configparser import SafeConfigParser
import pandas as pd
import numpy as np
import seaborn as sns

#### Read Configuration

In [ ]:
cfg = SafeConfigParser()

In [ ]:
cfg.read('conf/settings.ini')

In [ ]:
symbols = ['MSFT', 'LMT', 'OC']

#### Export Data

In [ ]:
data = {}

In [ ]:
def add_stock(symbol):
    data[symbol] = requests.get('https://%s/query' % cfg.get('AlphaVantage', 'host'), params={
        'function': 'TIME_SERIES_WEEKLY',
        'symbol': symbol,
        'apikey': cfg.get('AlphaVantage', 'api_key')
    })

In [ ]:
map(add_stock, symbols)

#### Clean Data

In [ ]:
df = {}

In [ ]:
def frame_data(symbol, dataset='Weekly Time Series'):
    df[symbol] = pd.DataFrame(json.loads(data[symbol].content)[dataset]).transpose().apply(lambda x: map(np.float, x))
    df[symbol].columns = ['open', 'high', 'low', 'close', 'volume']

In [ ]:
map(frame_data, symbols)

#### Crunch Numbers

In [ ]:
mdf = {}

In [ ]:
def get_growth_by_year(symbol, year, timerange=1, increment=52):
    rdf = df[symbol].tail(1 + (increment * year))
    rdf_avg = rdf.apply(lambda x: np.mean([x['open'], x['high'], x['low'], x['close']]), axis=1)
    c_rdf = rdf_avg.tail(1 + (increment * (year - timerange)))
    p_rdf = rdf_avg.head(1 + (increment * year))
    return rdf_avg, [p_rdf, c_rdf]

In [ ]:
years = 11

In [ ]:
for symbol in symbols:
    print '=' * 100
    for y in range(years)[::-1]:
        mdf[symbol], drange = get_growth_by_year(symbol, y+1)
        print os.linesep.join([
            'Symbol: %s' % symbol,
            'Price (%s): %.2f' % (drange[0].index[0], drange[0].values[0]),
            'Price (%s): %.2f' % (drange[1].index[0], drange[1].values[0]),
            'APY: %.2f%%' % ((drange[1].values[0] - drange[0].values[0]) / drange[0].values[0] * 100)
        ])
        print '-' * 100

#### Data Visualization

In [ ]:
%matplotlib inline

In [ ]:
map(lambda s: mdf[s].plot(label=s, legend=True), symbols)